# Batch Predictions
https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_batch_transform/tensorflow_cifar-10_with_inference_script/tensorflow-serving-cifar10-python-sdk.ipynb

and 

https://aws.amazon.com/blogs/machine-learning/performing-batch-inference-with-tensorflow-serving-in-amazon-sagemaker/

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Setup Input Data

In [ ]:
%store -r scikit_processing_job_name

In [ ]:
print(scikit_processing_job_name)

In [ ]:
# scikit_processing_job_s3_output_prefix = 'data'
print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_name))

In [ ]:
prefix_test = '{}/output/bert-test'.format(scikit_processing_job_name)

test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

In [ ]:
print(test_s3_uri)

!aws s3 ls $test_s3_uri/

# Setup Batch Transform Model

In [ ]:
%store -r training_job_name

In [ ]:
print(training_job_name)

In [ ]:
# !aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

In [ ]:
# !tar -xvzf ./model.tar.gz

In [ ]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

In [ ]:
from sagemaker.tensorflow.serving import Model

# If you change SAGEMAKER_TFS_DEFAULT_MODEL_NAME to something other than 'saved_model', you may see the dreaded ping error in the logs error
batch_env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'saved_model', # <== change this when using multi-model,
                                                     #     but watch out for the dreaded ping/ error 
                                                     #     if the model name doesn't exist
  'SAGEMAKER_TFS_ENABLE_BATCHING': 'true',
  'SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS': '50000',
  'SAGEMAKER_TFS_MAX_BATCH_SIZE': '16'
}

batch_model = Model(entry_point='batch_inference_tfrecord.py',
                    source_dir='src_batch_inference_tfrecord',
                    model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
                    role=role,
                    framework_version='2.1.0',
                    env=batch_env)

In [ ]:
batch_predictor = batch_model.transformer(instance_count=1, 
                                          strategy='MultiRecord', 
#                                          instance_type='local',                                          
                                          instance_type='ml.m4.xlarge',
                                          assemble_with='Line',
                                          max_concurrent_transforms=64,
                                          max_payload=1, # This is in Megabytes (not number of records)
                                          env=batch_env)

# Start Batch Predictions

In [ ]:
batch_predictor.transform(data=test_s3_uri, 
                          split_type='TFRecord',
                          content_type='application/x-tfexample')

In [ ]:
print('Waiting for batch prediction job: ' + batch_predictor.latest_transform_job.job_name)

batch_predictor.wait()

# Check Output Data

After the transform job has completed, download the output data from S3.

For each file "f" in the input data, we have a corresponding file "f.out" containing the predicted labels from each input row. 

We can compare the predicted labels to the true labels saved earlier.


In [ ]:
# Download the output data from S3 to local filesystem
batch_prediction_output_s3_uri = batch_predictor.output_path

# !mkdir -p ./batch_prediction_output

In [ ]:
%%bash 

aws s3 cp --recursive $batch_prediction_output_s3_uri/ ./batch_prediction_output

ls ./batch_prediction_output